# 1. Cargar las librerías

In [1]:
# Data libraries
import numpy as np
import pandas as pd
from functools import reduce

# AI libraries
from sklearn import preprocessing

# RL libraries
from gym import Env
from gym.spaces import Discrete, Box

# Generic libraries
import random

In [2]:
# Constantes del programa
DATA_PATH = "./datos/"
RANDOM_STATE=42
pd.options.display.max_columns = None

# Importes a jugar
BUDGET = 1000
BET_LIMIT = 10

# 2. Cargar el dataset en memoria

In [3]:
# Creamos lista ficheros a cargar
lista_ficheros=[DATA_PATH + 'SP1-{}-{}.csv'.format(x,x+1) for x in range(2005, 2022)]
# Cargamos todos los ficheros en una lista de tuplas Key-Value
temporadas=[(x[-13:-4],pd.read_csv(x, parse_dates=['Date'], dayfirst=True)) for x in lista_ficheros ]
# Añadimos campo temporada al dataset
temporadas=[ x[1].assign(Temporada=x[0]) for x in temporadas  ]
# Unimos todos los datasets en uno solo
liga=reduce(lambda x,y: pd.concat([x,y]), temporadas)
# Ajustamos el índice para ir de 1 a max, en lugar de repertirse el índice para cada temporada
liga.reset_index(drop=True, inplace=True)

# 3. Creamos el dataset para su análisis

In [4]:
# Reducimos las variables a cargar
lista_columnas=liga.columns[0:10].to_list()
lista_columnas.append('Temporada')
liga = liga[lista_columnas]
liga

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Temporada
0,SP1,2005-08-27,Alaves,Barcelona,0,0,D,0,0,D,2005-2006
1,SP1,2005-08-27,Ath Bilbao,Sociedad,3,0,H,0,0,D,2005-2006
2,SP1,2005-08-27,Valencia,Betis,1,0,H,0,0,D,2005-2006
3,SP1,2005-08-28,Ath Madrid,Zaragoza,0,0,D,0,0,D,2005-2006
4,SP1,2005-08-28,Cadiz,Real Madrid,1,2,A,0,1,A,2005-2006
...,...,...,...,...,...,...,...,...,...,...,...
6302,SP1,2022-02-06,Valencia,Sociedad,0,0,D,0,0,D,2021-2022
6303,SP1,2022-02-06,Barcelona,Ath Madrid,4,2,H,3,1,H,2021-2022
6304,SP1,2022-02-06,Betis,Villarreal,0,2,A,0,1,A,2021-2022
6305,SP1,2022-02-06,Real Madrid,Granada,1,0,H,0,0,D,2021-2022


In [5]:
# Label encoder de los equipos
leTeam=preprocessing.LabelEncoder()
leTeam.fit(liga['HomeTeam'].value_counts().index.to_list())
liga['HomeTeamCode']=leTeam.transform( liga['HomeTeam'])
liga['AwayTeamCode']=leTeam.transform( liga['AwayTeam'])

In [6]:
# Label encoder de la temporada
leTemporada=preprocessing.LabelEncoder()
leTemporada.fit(liga['Temporada'].value_counts().index.to_list())
liga['TemporadaCode']=leTemporada.transform(liga['Temporada'])

In [7]:
# Label encoder de los resultados
liga.loc[ liga['FTR']=='H', 'ResultadoFT'  ]=0
liga.loc[ liga['FTR']=='D', 'ResultadoFT'  ]=1
liga.loc[ liga['FTR']=='A', 'ResultadoFT'  ]=2

liga.loc[ liga['HTR']=='H', 'ResultadoHT'  ]=0
liga.loc[ liga['HTR']=='D', 'ResultadoHT'  ]=1
liga.loc[ liga['HTR']=='A', 'ResultadoHT'  ]=2

In [8]:
# Se aumentan las columnas con el resultado de los últimos 5 partidos, tanto a la media parte como al final del partido
for x in [1,2,3,4,5]:
    liga['ResultadoHT-'+str(x)]=liga.groupby(['HomeTeam'])['ResultadoHT'].shift(x)
    liga['ResultadoFT-'+str(x)]=liga.groupby(['HomeTeam'])['ResultadoFT'].shift(x)

In [9]:
# Se subsituyen los NaN por valores en rango muy distinto al valor de la variable para que el algorimo sepa diferenciarlo.
liga.fillna(-999, inplace=True)

In [10]:
# Mostramos las columnas existentes
liga.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Temporada', 'HomeTeamCode', 'AwayTeamCode',
       'TemporadaCode', 'ResultadoFT', 'ResultadoHT', 'ResultadoHT-1',
       'ResultadoFT-1', 'ResultadoHT-2', 'ResultadoFT-2', 'ResultadoHT-3',
       'ResultadoFT-3', 'ResultadoHT-4', 'ResultadoFT-4', 'ResultadoHT-5',
       'ResultadoFT-5'],
      dtype='object')

In [11]:
# Eliminanos las columnas que:
# - Corresponden a variables que han sido encoded
# - Son variables las cuales no dispondremos en el momento de la predicción, como los resultados a mitad de partido o el resultado final
lista_borrar=liga.columns[0:11].to_list()
lista_borrar.append('ResultadoFT')
lista_borrar.append('ResultadoHT')

In [12]:
X=liga.drop(lista_borrar, axis=1)
Y=liga['ResultadoFT']
X_head=liga[['Date','HomeTeam','AwayTeam']]

# 4. Creamos el gym para la simulación con LR

In [13]:
class LaLigaEnv(Env):
    
    def initBudget(self):
        # fixing the budget
        self.budget = BUDGET
        # fixing the bet amount
        self.betlimit = BET_LIMIT
        # current state is initial budget
        self.state = self.budget
    
    def initSeason(self):
        # select season from data
        alldata = self.matchs
        
        # Select season
        minSeason = alldata['TemporadaCode'].min()
        maxSeason = alldata['TemporadaCode'].max()
        self.season = random.randint(minSeason, maxSeason)
        
        # Select season matches
        condition = alldata['TemporadaCode']==self.season
        self.seasonMatchs = alldata[condition]
        
        # Select first season match
        self.seasonCurrMatch = self.seasonMatchs.head(1)

    def setObservationSpace(this):
        
        self.observation_space = Box(self.seasonCurrMatch)
        

    def __init__(self, matchs, results):
        
        # Store data
        self.matchs = matchs
        self.results = results
        
        # Init budget
        self.initBudget()
        
        # Select season
        self.initSeason()
        
        # 0 = Home 1 = Draw 2 = Away 3 = Not Betting
        self.action_space = Discrete(4)
        
        #self.observation_space = 
        
    def win(self, rate):
        self.reward = rate * self.betlimit
        self.budget += (self.reward - self.betlimit)
    
    def loose(self):
        self.reward = 0
        self.budget -= self.betlimit
        
    def notBetting(self):
        self.reward = self.betlimit
        
    def step(self, action):
        
        # Apply action
        # TODO
        result = {
          0: self.notBetting(),
          1: self.notBetting(),
          2: self.notBetting(),
          3: self.notBetting()
        }[action]
   
        # Calculate reward
        self.reward = random.randint(0,10)
        # TODO

        # Update budget
        self.budget = self.budget - self.betlimit + self.reward
        if (self.budget < 0):
            self.budget = 0
        
        # Advance to next match
        newIndex = self.seasonCurrMatch.index[0]+1
        self.seasonCurrMatch = self.seasonMatchs.loc[newIndex:newIndex]
        
        # Check if done
        iSCMTC = self.seasonCurrMatch['TemporadaCode'].values[0]
        self.done = (self.budget <= 0) | (iSCMTC != self.season)
       
        # Update observation
        # TODO
        
        # Set placeholder for info
        # TODO
        self.info = { self.season, self.seasonCurrMatch.to_string() }
        
        # Return 
        return self.state, self.reward, self.done, self.info
    
    def render(self):
        pass

    def reset(self):
        self.initBudget()
        self.initSeason()
        return self.state

In [14]:
env = LaLigaEnv(X, Y)
episodes=10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False

    while not done:
        env.render()
        action = random.randint(0,3)
        n_state, reward, done, info = env.step(action)
    print('Episode:{} Budget:{}'.format(episode, env.budget))

Episode:1 Budget:0
Episode:2 Budget:0
Episode:3 Budget:0
Episode:4 Budget:0
Episode:5 Budget:0
Episode:6 Budget:0
Episode:7 Budget:0
Episode:8 Budget:0
Episode:9 Budget:0
Episode:10 Budget:0
